In [321]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier

In [322]:
train = pd.read_csv('amexNA.csv')
train.drop(['Unnamed: 0'],axis=1,inplace=True)
train.head()
test=pd.read_csv('amextestnA.csv')
test.drop(['Unnamed: 0'],axis=1,inplace=True)

In [354]:
test=pd.read_csv('Evaluation_dataset.csv')

In [357]:
test=test.replace('missing',np.nan)
test=test.replace('na',np.nan)
test=test.replace('N/A',np.nan)

for sl in range(1,47):
    test['mvar'+str(sl)] = test['mvar'+str(sl)].astype(float)
    
test['mvar47'].loc[test['mvar47'] == 'L'] = 0
test['mvar47'].loc[test['mvar47'] == 'C'] = 1

C:\Users\iramaniparth\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [323]:
ITERATIONS = 3 # 1000
TRAINING_SIZE = 80000*0.75 # 20000000
TEST_SIZE = 80000*0.25

In [324]:
train.head()
train0=np.array(train)
mn=np.mean(train0, axis=0)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train0[:,1:48], train0[:,48], random_state=0, test_size = 0.25)
print(train0.shape)
train0[:,48]

(80000, 49)


array([0., 1., 1., ..., 1., 0., 1.])

In [325]:
# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = xgb.XGBClassifier(
        n_jobs = 1,
        objective = 'binary:logistic',
        eval_metric = 'auc',
        silent=1,
        tree_method='approx',
        early_stopping_rounds=10,
        eval_set= [(X_test, y_test)]
    ),
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'min_child_weight': (0, 10),
        'max_depth': (0, 50),
        'max_delta_step': (0, 20),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'colsample_bylevel': (0.01, 1.0, 'uniform'),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'gamma': (1e-9, 0.5, 'log-uniform'),
        'min_child_weight': (0, 5),
        'n_estimators': (50, 100),
        'scale_pos_weight': (1e-6, 500, 'log-uniform')
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")

In [326]:
result_xgb = bayes_cv_tuner.fit(train0[:,1:48], train0[:,48], callback=status_print)

Model #1
Best ROC-AUC: 0.8342
Best params: {'colsample_bylevel': 0.4160029192647807, 'colsample_bytree': 0.7304484857455519, 'gamma': 0.13031389926541354, 'learning_rate': 0.042815319280763466, 'max_delta_step': 13, 'max_depth': 21, 'min_child_weight': 2, 'n_estimators': 87, 'reg_alpha': 5.497557739289786e-07, 'reg_lambda': 0.05936070635912049, 'scale_pos_weight': 0.060830282487222144, 'subsample': 0.13556548021189216}

Model #2
Best ROC-AUC: 0.8371
Best params: {'colsample_bylevel': 0.8390144719977516, 'colsample_bytree': 0.8844821246070537, 'gamma': 4.358684608480795e-07, 'learning_rate': 0.7988179462781242, 'max_delta_step': 17, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 68, 'reg_alpha': 0.0005266983003701547, 'reg_lambda': 276.5424475574225, 'scale_pos_weight': 0.3016410771843142, 'subsample': 0.9923710598637134}

Model #3
Best ROC-AUC: 0.8371
Best params: {'colsample_bylevel': 0.8390144719977516, 'colsample_bytree': 0.8844821246070537, 'gamma': 4.358684608480795e-07, '

In [358]:
test0=np.array(test.iloc[:,1:])
ynew = result_xgb.predict_proba(test0)

In [359]:
d=pd.DataFrame()
d['application_key']=test['application_key']
d['0']=ynew[:,0]
d['1']=ynew[:,1]

In [360]:
y=result_xgb.predict(test0)

d['pred']=y

d['max']=d[['0', '1']].max(axis=1)

d['max']=(d['max']).apply(lambda x: round(x,2))

d['max2']=d['max']-0.001*d['pred']
d = d.sort_values(by = ['max2'], ascending = False)

sub1 = d[['application_key', 'pred']]

sub1.to_csv('The_Miner_League_IITGuwahati_final_submission.csv', header = False, index = False)

C:\Users\iramaniparth\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [346]:
d.loc[d['0'] > 0.975].shape

(8256, 6)

In [348]:
d.loc[d['1'] > 0.6].shape

(605, 6)

In [75]:
# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = lgb.LGBMRegressor(
        objective='binary',
        metric='auc',
        n_jobs=1,
        verbose=0,
        use_missing=True
    ),
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'num_leaves': (1, 100),      
        'max_depth': (0, 50),
        'min_child_samples': (0, 50),
        'max_bin': (100, 1000),
        'subsample': (0.01, 1.0, 'uniform'),
        'subsample_freq': (0, 10),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'min_child_weight': (0, 10),
        'subsample_for_bin': (100000, 500000),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'scale_pos_weight': (1e-6, 500, 'log-uniform'),
        'n_estimators': (50, 100),
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)

# Fit the model
result_lgb = bayes_cv_tuner.fit(train0[:,1:45], train0[:,45], callback=status_print)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [69]:
test0=np.array(test.iloc[:,1:])
ynew = result_lgb.predict(test0)

In [70]:
d=pd.DataFrame()
d['application_key']=test['application_key']
d['1']=pd.DataFrame(ynew)
d['0']=1-d['1']

In [71]:
d.loc[d['1']>d['0'], 'pred'] = 1
d.loc[d['0']>d['1'], 'pred'] = 0

In [72]:
#y=result_lgb.predict(test0)

#d['pred']=y

d['max']=d[['0', '1']].max(axis=1)

d['max']=d['max'].apply(lambda x: round(x, 2))

d['max2']=d['max']-0.001*d['pred']
d = d.sort_values(by = ['max2'], ascending = False)

sub1 = d[['application_key', 'pred']]

sub1.to_csv('The_miner_league_IITGuwahati_23.csv', header = False, index = False)

In [13]:
# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = GradientBoostingClassifier(),
    search_spaces = {
        'learning_rate': (10**-5, 10, 'log-uniform'),     
        'max_depth': (0, 50),
        'subsample': (0.01, 1.0, 'uniform'),
        'n_estimators': (50, 100),
        'min_samples_leaf': (1, 100),
        'min_samples_split': (2, 100),
        'max_features': (1, 44),
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)

# Fit the model
result_gbm = bayes_cv_tuner.fit(train0[:,1:45], train0[:,45], callback=status_print)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [74]:
test0=np.array(test.iloc[:,1:])
ynew = result_gbm.predict_proba(test0)

In [75]:
d=pd.DataFrame()
d['application_key']=test['application_key']
d['0']=ynew[:,0]
d['1']=ynew[:,1]

In [76]:
y=result_gbm.predict(test0)

d['pred']=y

d['max']=d[['0', '1']].max(axis=1)

d['max']=d['max'].apply(lambda x: round(x, 2))

d['max2']=d['max']-0.001*d['pred']
d = d.sort_values(by = ['max2'], ascending = False)

sub1 = d[['application_key', 'pred']]

sub1.to_csv('The_miner_league_IITGuwahati_24.csv', header = False, index = False)